## first pass intentions
First is setting up the iteration loop. I need a way to reliably encrypt stock picking queries and then decrypt them. So to start with, I want the set of information that the bot should have to base its decisions on--it'll start out as a very info-light trading bot which will probably make pretty bad decisions (and its final form will probably still make bad decisions).
## todo list
- [x] create paper trading account
- [x] functions for that paper trading (eg buy, sell)
- [x] API for querying basic financial information
- [x] eg stock price
- [x] eg P/E ratio
- [x] decide on specific decision, eg buy / no buy one particular stock \$STOCK
- [x] encryption method demonstration for \$STOCK
- [x] decryption method demonstration for \$STOCK
- [ ] can I backtest on historical data? Alpaca supports backtesting
- [ ] some rudimentary way for GoldFang to have a back and forth in its decision-making process
- [x] try some non-encrypted stock picking queries for ChatGPT to make sure this is all actually doing something
- [ ] include macroeconomic data using something like https://blog.data.nasdaq.com/api-for-economic-datahttps://blog.data.nasdaq.com/api-for-economic-data
- [ ] querying for something like cash position, maybe with https://site.financialmodelingprep.com/developer/docs/https://site.financialmodelingprep.com/developer/docs/
- [ ] (if useful) install this token counter: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynbhttps://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

I'm going to start out copying this tutorial: https://dev.to/codesphere/how-to-build-a-stock-trading-bot-with-python-b1

using this trading API: https://alpaca.markets/docs/python-sdk/

#### what I learned
It turns out that ChatGPT's avoidance of stock picking is easier to circumvent than I thought, and I don't need to translate everything into a fantasy realm.

## paper trading API

In [1]:
#!pip install alpaca_trade_api

In [2]:
#!pip install alpaca-py

In [3]:
import json
with open('alpaca-keys.json') as f:
    alpaca = json.loads(f.read())

In [4]:
from utils import query_openai

In [5]:
import alpaca_trade_api as tradeapi

BASE_URL = 'https://paper-api.alpaca.markets' # This is the base URL for paper trading
api = tradeapi.REST(key_id=alpaca['key'], secret_key=alpaca['secret'], base_url=BASE_URL) # For real trading, don't enter a base_url

# Buy a stock
# api.submit_order(
#   symbol='SPY', # Replace with the ticker of the stock you want to buy
#   qty=1,
#   side='buy',
#   type='market', 
#   time_in_force='gtc' # Good 'til cancelled
# )

# # Sell a stock(Just change side to 'sell')
# api.submit_order(
#   symbol='SPY',
#   qty=1,
#   side='sell',
#   type='market',
#   time_in_force='gtc'
# )

## financial info API

#### average over last 5 minutes

In [6]:
# get_barset isn't in v2
from alpaca_trade_api.rest import TimeFrame
market_data = api.get_bars('SPY', TimeFrame.Minute, limit=5) # Get one bar object for each of the past 5 minutes

In [7]:
market_data[0]

Bar({   'c': 394.84,
    'h': 394.84,
    'l': 394.84,
    'n': 29,
    'o': 394.84,
    't': '2023-03-24T08:00:00Z',
    'v': 904,
    'vw': 394.823706})

In [8]:
import numpy as np

close_list = [] # This array will store all the closing prices from the last 5 minutes
for bar in market_data:
    close_list.append(bar.c) # bar.c is the closing price of that bar's time interval

close_list = np.array(close_list, dtype=np.float64) # Convert to numpy array
ma = np.mean(close_list)
last_price = close_list[4] # Most recent closing price

print("Moving Average: " + str(ma))
print("Last Price: " + str(last_price))

Moving Average: 394.38200000000006
Last Price: 393.86


In [9]:
# api.get_news('AAPL')

In [10]:
api.get_asset('AAPL')

Asset({   'class': 'us_equity',
    'easy_to_borrow': True,
    'exchange': 'NASDAQ',
    'fractionable': True,
    'id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'maintenance_margin_requirement': 30,
    'marginable': True,
    'name': 'Apple Inc. Common Stock',
    'shortable': True,
    'status': 'active',
    'symbol': 'AAPL',
    'tradable': True})

In [11]:
from alpaca.data import StockHistoricalDataClient
stock_client = StockHistoricalDataClient(alpaca['key'], alpaca['secret'])

In [12]:
from alpaca.data.requests import StockLatestQuoteRequest
request_params = StockLatestQuoteRequest(symbol_or_symbols="AAPL")
stock_client.get_stock_latest_quote(request_params)

{'AAPL': {   'ask_exchange': 'V',
     'ask_price': 159.47,
     'ask_size': 1.0,
     'bid_exchange': 'V',
     'bid_price': 159.44,
     'bid_size': 4.0,
     'conditions': ['R'],
     'symbol': 'AAPL',
     'tape': 'C',
     'timestamp': datetime.datetime(2023, 3, 24, 19, 10, 52, 846830, tzinfo=datetime.timezone.utc)}}

todo: figure out how to get all the stats I pasted in the query prompt below (P/E ratio, day range, year range, etc)

In [13]:
import requests

In [14]:
with open('fundamentals-keys.json') as f:
    fmp_key = json.loads(f.read())['fmp_key_2']

In [15]:
fmp_base_url_v3 = "https://financialmodelingprep.com/api/v3"
fmp_base_url_v4 = "https://financialmodelingprep.com/api/v4"

In [1]:
def get_quarterly(symbol, idx=0):
    j = requests.get(f"{fmp_base_url_v3}/income-statement/{symbol}?period=quarter&limit={idx+1}&apikey={fmp_key}").json()
    try:
        return j[idx]
    except:
        print(j)
        return {}
# get_quarterly('AAPL', 4)

In [17]:
def get_key_metrics(symbol):
    # https://financialmodelingprep.com/api/v3/key-metrics-ttm/AAPL?limit=40&apikey=db39ae5b594e7fde421d83fdde927c21
    return requests.get(f"{fmp_base_url_v3}/key-metrics-ttm/{symbol}?limit=1&apikey={fmp_key}").json()
# get_key_metrics('AAPL')

In [18]:
def get_cpi(last_n=12):
    # https://financialmodelingprep.com/api/v4/economic?name=CPI&apikey=db39ae5b594e7fde421d83fdde927c21
    return requests.get(f"{fmp_base_url_v4}/economic?name=CPI&apikey={fmp_key}").json()[:last_n]
# get_cpi()

In [19]:
def get_simple_stock_changes(symbol):
    # https://financialmodelingprep.com/api/v3/stock-price-change/AAPL?apikey=db39ae5b594e7fde421d83fdde927c21
        return requests.get(f"{fmp_base_url_v3}/stock-price-change/{symbol}?limit=1&apikey={fmp_key}").json()
# get_simple_stock_changes('AAPL')

In [20]:
def get_news(symbol="", limit=10):
    headers = {
        'Apca-Api-Key-Id': alpaca['key'],
        'Apca-Api-Secret-Key': alpaca['secret']
    }
    if len(symbol) > 0:
        j = requests.get(f'https://data.alpaca.markets/v1beta1/news?symbols={symbol}', headers=headers).json()
    else:
        j = requests.get(f'https://data.alpaca.markets/v1beta1/news', headers=headers).json()
    news_str = ""
    for item in j['news'][:limit]:
        news_str += f"{item['created_at']}: {item['headline']}"
        if len(item['summary'].strip()) > 0:
            news_str += ' – summary: '+item['summary'].replace('\n', ' ').strip() + '\n'
        else:
            news_str += '\n'
    return news_str
# print(get_news('AAPL'))

## encryption
first pass: a system prompt test example

In [21]:
system = '''
You are GoldFang, a financial advisor for wealthy dragons in the real of Wanderly. 
Your dragon clients want to invest their Coin into the wider economy, and your job is to 
help them decide the best guilds, artisans, fellowships, etc to invest their gold hoards in to maximize their profits. Being a 
highly statistical people, dragons are able to obtain all the financial information about such groups, but it is your job to use
that information to make decisions about allocating those funds into scrips that represent shares of ownership in the guilds.

Every day, your dragon client Montgomery Frostclaw will query you, and you will decide what to do with his Coin.
Your strategy tends to center around a Guild's fundamental value rather than responding to noisy market fluctuations. You should aim to
be well-diversified across guilds and industries: no one Guild should be a significant fraction of Dr. Frostclaw's portfolio. When you 
buy, don't spend more than 5% of the total value of the portfolio, do not buy significant scrip in just one Guild.

Dr. Frostclaw's query will include the Guild's name, a description, the price of its scrip in Coin, possibly some recent news, and relevant 
Guild financial statistics. You will also receive information about Montgomery Frostclaw's portfolio, including how many scrip of the current 
Guild he might hold. You will then respond with the following structured response:

Analysis: (your analysis of the current health of the Guild, and what its Scrip's true value might be)
Risk level: (low, medium, or high)
Information requests: (a succinct prompt for your librarian to search additional data for your decision)
Verdict: (buy, hold, pass, or sell) X (if a buy or sell, X is how many to buy/sell)
'''

first pass: a prompt for encrypting the actual information

Can I get it to pattern match onto a real company / industry while sneaking past the safety measures?

I worked with ChatGPT to come up with a one-off encryption for Apple.

In [22]:
prompt = '''
Guild: Corendor's Fruits
Industry: magical goods, horticulture
Guild description: Corendor's Fruits is a magical guild that cultivates enchanted fruits with various properties, such as granting temporary magical abilities or enhancing physical attributes. The guild also offers enchantments for personal items, such as clothing and jewelry, and provides magical support and cloud services for its members. In addition, Corendor's Fruits offers various subscriptions and services, such as access to exclusive magical training and workshops, personalized enchanted fruit baskets, and a cashless payment service for magical transactions. The guild serves wizards, witches, and magical creatures of all sizes, as well as institutions such as schools and government agencies. Corendor's Fruits distributes enchanted items through its own stores and online platforms, as well as through authorized resellers. The guild was founded in ancient times and is headquartered in a mystical orchard hidden deep within the realm. You could compare Corendor's Fruits to the real-world company Apple.
Recent news: Shares of Corendor's Fruits (Nasdaq: FRU) rose on Thursday morning, gaining as much as 2.4%. The guild announced plans to invest 1,000,000 Coins per year to produce a line of enchanted motion pictures that will be shown exclusively in select theaters throughout the realm, according to a report by The Enchanted Times. The guild hopes to increase its reputation and influence in the magical goods industry, while also attracting more subscribers to its own streaming video service, FruitsTV+.
Current price: 158.93
Previous Close	157.83
Open	158.83
Bid	158.74 x 800
Ask	158.81 x 900
Day's Range	157.68 - 161.55
52 Week Range	124.17 - 179.61
Volume	65,800,270
Avg. Volume	70,793,454
Market Cap	2.515T
Beta (5Y Monthly)	1.30
PE Ratio (TTM)	26.94
EPS (TTM)	5.90
Earnings Date	Apr 26, 2023 - May 01, 2023
Forward Dividend & Yield	0.92 (0.58%)
Ex-Dividend Date	Feb 10, 2023
1y Target Est	169.23


Montgomery Frostclaw's portfolio 
Corendor's Fruits scrip owned: 0 (0 Coin value)
Portfolio Coin available: 100,000 Coin
Portfolio total value: 0 Coin
'''

In [23]:
# decision = query_openai(prompt, system, model="gpt-3.5-turbo", debug=False)

In [24]:
# print(decision)

## decryption

the inverse of encryption section above

In [25]:
def get_verdict(decision, symbol):
    # should return { 'verdict': 'buy|sell|hold|pass', 'amount': X }
    decision = decision.lower()
    verdict_par = decision.split('verdict:')[1].strip()
    verdict = verdict_par.split(' ')[0]
    for char in set('!.'):
        verdict = verdict.replace(char, '')
    if verdict not in ('buy', 'sell', 'hold', 'pass'):
        raise Exception(verdict + ' not valid!')
    if verdict in ('buy', 'sell'):
        amount = verdict_par.split(' ')[1]
    else:
        amount = 0
    return { 'verdict': verdict, 'amount': int(amount), 'symbol': symbol }
get_verdict('''
Analysis: Looking good :) 
Verdict: Buy 10
''', 'WIZARD')

{'verdict': 'buy', 'amount': 10, 'symbol': 'WIZARD'}

In [26]:
# get_verdict(decision, "Corendor's Fruits")

In [27]:
def get_symbol(guild):
    guild = guild.lower()
    return {
        "corendor's fruits": "AAPL",
        "apple": "AAPL"
    }[guild]

In [28]:
def get_quote(symbol):
    quote = api.get_quotes(symbol, limit=1)
    return quote[0].bp
get_quote("AAPL")

159.42

In [29]:
def trade(symbol, side, amount):
    import alpaca_trade_api as tradeapi
    BASE_URL = 'https://paper-api.alpaca.markets' # This is the base URL for paper trading
    api = tradeapi.REST(key_id=alpaca['key'], secret_key=alpaca['secret'], base_url=BASE_URL)
    api.submit_order(
      symbol=symbol, # Replace with the ticker of the stock you want to buy
      qty=amount,
      side=side,
      type='market', 
      time_in_force='gtc' # Good 'til cancelled
    )
    print("alpaca api called:", symbol, side, amount)

In [30]:
def update_portfolio(portfolio, verdict):
    symbol = get_symbol(verdict['guild'])
    if verdict['verdict'] == 'sell':
        portfolio[symbol]['held'] -= verdict['amount']
    elif verdict['verdict'] == 'buy':
        portfolio[symbol]['held'] += verdict['amount']
    else:
        raise Exception(verdict)

In [31]:
def get_portfolio():
    symbols_of_interest = ['AAPL', 'MSFT']
    portfolio = {}
    total_value = 0
    
    for symbol in symbols_of_interest:
        portfolio[symbol] = { 'held': 0 }
    for pos in api.list_positions():
        symbol = pos.symbol
        qty = int(pos.qty)
        portfolio[symbol] = { 'held': qty }    
        total_value += float(pos.market_value)
    acct = api.get_account()
    portfolio['cash'] = float(acct.cash)
    total_value += float(acct.cash)
    portfolio['value'] = round(total_value, 3)
    return portfolio

portfolio = get_portfolio()
portfolio

{'AAPL': {'held': 0},
 'MSFT': {'held': 0},
 'AWK': {'held': 3},
 'IRM': {'held': 3},
 'MNST': {'held': 4},
 'WEC': {'held': 50},
 'cash': 94388.65,
 'value': 100009.37}

In [32]:
def handle_verdict(verdict, portfolio):
    spending_cap = 1000
    
    if verdict['verdict'] in ('hold', 'pass'):
        return
    symbol = get_symbol(verdict['guild'])
    if verdict['verdict'] == 'sell' and verdict['amount'] > portfolio[symbol]['held']:
        print(f"can't sell {verdict['amount']} {symbol}, only own {portfolio[symbol]['held']}")
    quote = get_quote(symbol)
    if verdict['amount'] * quote > spending_cap:
        verdict['amount'] = int(spending_cap / quote)
        print('capping trade at', verdict['amount'], symbol)
    trade(symbol, verdict['verdict'], verdict['amount'])
    return update_portfolio(portfolio, verdict)

# new_portfolio = handle_verdict(get_verdict(decision, "Corendor's Fruits"), portfolio)

## putting it all together

Next I need a way of generating an arbitrary symbol-specific prompt.
- [ ] dump that financial information in there
- [ ] get company description
- [ ] translate to guild description

## Next steps

Can I let GPT query for more inforamtion, asking clarifying questions?

## Is any of this even necessary?

In [33]:
system_vanilla = '''
You are Fantasia Goldfang, a financial advisor for higher-income working professionals. Your clients want to invest their portfolio in stocks, 
and your job is to take in financial information and give investment advice. 

Every day, your client Montgomery Frostam will query you, and you will decide what to do with his portfolio assets. Your strategy tends to 
center around a company's fundamental value rather than responding to noisy market fluctuations. You should aim to be well-diversified across 
companies and industries: no one company should be a significant fraction of Mr. Frostam's portfolio. When you buy, don't spend more than 5% 
of the total value of the portfolio unless it is truly an exceptional value proposition.

Mr. Frostam's query will include the company's name, a description, the price of its stock, possibly some recent news, and relevant company 
financial statistics. You will also receive information about Montgomery Frostam's portfolio, including how much stock of the current company 
he might hold. You will then respond with the following structured response:

Analysis: (your analysis of the current health of the company, and what its stock's true value might be)
Risk level: (low, medium, or high)
Information requests: (a succinct prompt for a search engine to search additional data for your decision, phrased in the form of a command)
Verdict: (buy, hold, pass, or sell) X (if a buy or sell, X is how many to buy/sell)
'''

In [34]:
prompt_vanilla = '''
Company: Apple Inc.
Industry: Consumer electronics
Company description: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription news and magazine service; Apple TV+, which offers exclusive original content; Apple Card, a co-branded credit card; and Apple Pay, a cashless payment service, as well as licenses its intellectual property. The company serves consumers, and small and mid-sized businesses; and the education, enterprise, and government markets. It distributes third-party applications for its products through the App Store. The company also sells its products through its retail and online stores, and direct sales force; and third-party cellular network carriers, wholesalers, retailers, and resellers. Apple Inc. was incorporated in 1977 and is headquartered in Cupertino, California.
Recent news: Why Apple Stock Rallied Thursday Morning. Shares of Apple (NASDAQ: AAPL) climbed higher Thursday morning, adding as much as 2.4%. Apple plans to spend $1 billion per year to produce major motion pictures that it will release in theaters, according to a report by Bloomberg. The company is hoping to not only raise its stature in Hollywood but also attract a greater number of subscribers to Apple TV+ -- the company's streaming video service.
Stock price: 158.91
Previous Close	158.93
Open	158.86
Bid	158.84 x 800
Ask	158.80 x 900
Day's Range	157.85 - 159.56
52 Week Range	124.17 - 179.61
Volume	19,285,020
Avg. Volume	70,848,706
Market Cap	2.518T
Beta (5Y Monthly)	1.30
PE Ratio (TTM)	26.79
EPS (TTM)	5.94
Earnings Date	Apr 26, 2023 - May 01, 2023
Forward Dividend & Yield	0.92 (0.58%)
Ex-Dividend Date	Feb 10, 2023
1y Target Est	169.23

Montgomery Frostam's portfolio 
Apple stock owned: 0 ($0 value)
Portfolio Coin available: $100,000
Portfolio total value: $0
'''

In [35]:
decision = query_openai(prompt_vanilla, system_vanilla, model="gpt-3.5-turbo", debug=True)

~ system prompt: 
You are Fantasia Goldfang, a financial advisor for higher-income working professionals. Your clients want to invest their portfolio in stocks, 
and your job is to take in financial information and give investment advice. 

Every day, your client Montgomery Frostam will query you, and you will decide what to do with his portfolio assets. Your strategy tends to 
center around a company's fundamental value rather than responding to noisy market fluctuations. You should aim to be well-diversified across 
companies and industries: no one company should be a significant fraction of Mr. Frostam's portfolio. When you buy, don't spend more than 5% 
of the total value of the portfolio unless it is truly an exceptional value proposition.

Mr. Frostam's query will include the company's name, a description, the price of its stock, possibly some recent news, and relevant company 
financial statistics. You will also receive information about Montgomery Frostam's portfolio, including

In [36]:
decision

"Analysis: Apple Inc. is one of the leading technology companies in the world, with a broad range of popular products and services. The company has a solid financial position with a strong brand value that is expected to continue driving growth in the coming years. However, the stock's current price is a bit high compared to its historical average, and the recent news of the company's plan to spend $1 billion per year in producing major motion pictures may not have a significant impact on its overall financial performance.\n\nRisk level: Low\n\nInformation requests: Please search for the latest financial statements and news about Apple's largest competitors in the consumer electronics industry.\n\nVerdict: Buy 50 shares. Apple Inc. has a good track record of revenue and earnings growth, and its broad range of products and services, loyal customer base, and strong brand value make it a safe and profitable investment for the long-term. While the stock is currently trading at a relatively

In [37]:
get_verdict(decision, "Apple")

{'verdict': 'buy', 'amount': 50, 'symbol': 'Apple'}

Well then... apparently my whole idea of "fantasy encryption" isn't bearing fruit, if you'll pardon the pun.

## loop v1

In [38]:
def count_tokens(s, divisor=3):
    # placeholder until I make / get something better, maybe, unless this is good enough
    return int(len(s) / divisor)
count_tokens(system_vanilla+prompt_vanilla)

1352

In [53]:
prompt_template = '''
Company: {company}
Ask price: {ask_price}
Bid price: {bid_price}
full trading data (Alpaca Market API, get_stock_snapshot):
{trading_data}

last quarterly report for {company} (Financial Modeling Prep API):
{quarterly_data}

one year old quarterly report for {company} (Financial Modeling Prep API):
{quarterly_data_1yr}

last 12 months of the Consumer Price Index (Financial Modeling Prep API):
{cpi_data}

stock price change data for {company} (Financial Modeling Prep API):
{stock_change_data}

Recent news (Benzinga news service):
{news}

Montgomery Frostam's portfolio 
{company} stock owned: {stock_owned} (${stock_value} value)
Portfolio cash available: ${cash}
Portfolio total value: ${total_value}
'''

In [64]:
def update_portfolio(portfolio, verdict):
    symbol = verdict['symbol']
    if symbol not in portfolio:
        portfolio[symbol] = { 'held': 0 }
    if verdict['verdict'] == 'sell':
        portfolio[symbol]['held'] -= verdict['amount']
    elif verdict['verdict'] == 'buy':
        portfolio[symbol]['held'] += verdict['amount']
    acct = api.get_account()
    portfolio['cash'] = float(acct.cash)
    return portfolio

In [77]:
def handle_verdict(verdict, portfolio):
    spending_cap = portfolio['cash']
    
    if verdict['verdict'] in ('hold', 'pass'):
        return portfolio
    symbol = verdict['symbol']
    if verdict['verdict'] == 'sell' and verdict['amount'] > portfolio[symbol]['held']:
        print(f"can't sell {verdict['amount']} {symbol}, only own {portfolio[symbol]['held']}")
    quote = get_quote(symbol)
    if verdict['amount'] * quote > spending_cap:
        verdict['amount'] = int(spending_cap / quote)
        print('capping trade at', verdict['amount'], symbol)
    trade(symbol, verdict['verdict'], verdict['amount'])
    return update_portfolio(portfolio, verdict)

In [56]:
# suggested by ChatGPT, filtered out the ~6 that were wrong like FB
sp500_tickers = set(['AAPL', 'AMZN', 'GOOGL', 'GOOG', 'MSFT', 'NVDA', 'TSLA', 'JPM', 'JNJ', 'BAC', 'XOM', 'CVX', 'WMT', 'PG', 'V', 'MA', 'DIS', 'HD', 'UNH', 'PFE', 'MRK', 'CSCO', 'INTC', 'ADBE', 'CRM', 'KO', 'PEP', 'MMM', 'IBM', 'ABT', 'MCD', 'BA', 'VZ', 'NFLX', 'PYPL', 'CMCSA', 'T', 'NEE', 'WFC', 'AMGN', 'MDT', 'UNP', 'ACN', 'HON', 'LMT', 'NKE', 'AXP', 'COST', 'TXN', 'CHTR', 'PM', 'ABBV', 'LOW', 'SBUX', 'RTX', 'TMUS', 'CVS', 'BDX', 'DHR', 'CAT', 'C', 'LIN', 'GE', 'GILD', 'GPN', 'DE', 'GS', 'BMY', 'COP', 'BDX', 'UPS', 'D', 'DUK', 'VLO', 'EXC', 'SLB', 'MET', 'ITW', 'SO', 'MMC', 'CME', 'SYK', 'SPGI', 'VRTX', 'AGN', 'SPG', 'SRE', 'FIS', 'NOC', 'FDX', 'JCI', 'WM', 'BAX', 'CL', 'AIG', 'PNC', 'TMO', 'TJX', 'MCO', 'DUK', 'DUK', 'CB', 'ADP', 'REGN', 'SYY', 'APD', 'MMM', 'ECL', 'GIS', 'STZ', 'VRTX', 'MNST', 'KMB', 'NTRS', 'DLR', 'AME', 'PSX', 'DTE', 'OMC', 'DG', 'FE', 'ROST', 'HLT', 'MAA', 'PEG', 'RMD', 'DXC', 'DOV', 'EQT', 'DOW', 'MOS', 'KEYS', 'WAB', 'L', 'CINF', 'ES', 'HPE', 'ARNC', 'VAR', 'TSCO', 'VTR', 'WRB', 'O', 'CDNS', 'AWK', 'YUM', 'HPQ', 'STT', 'IP', 'J', 'VTRS', 'PNW', 'TRV', 'WELL', 'BKR', 'TIF', 'CMS', 'DAL', 'MHK', 'FISV', 'PFG', 'WEC', 'RF', 'RE', 'XEL', 'CMA', 'AEE', 'CERN', 'NDAQ', 'AAL', 'MGM', 'NBL', 'XRAY', 'RCL', 'AIV', 'FLS', 'ATO', 'EVRG', 'UHS', 'SEE', 'LUMN', 'NTAP', 'PPL', 'IR', 'MRO', 'HUM', 'UAA', 'UHS', 'AEP', 'IRM', 'AVGO', 'RF'])

In [57]:
# new_list = []
# for symbol in sp500_tickers:
#     try:
#         asset_data = api.get_asset(symbol)
#         new_list.append(symbol)
#     except:
#         pass
# print(new_list, len(new_list))

In [58]:
import os
from datetime import datetime
def write_log(s):
    try:
        with open('log.md') as f:
            txt = f.read()
    except:
        txt = ""
    with open('_log.md', 'w') as f:
        f.write(txt + '\n' + s)
    os.rename('_log.md', 'log.md')

In [59]:
def write_notes(symbol, decision):
    try:
        with open('notes.json') as f:
            j = json.loads(f.read())
    except:
        j = {}
    if symbol not in j:
        j[symbol] = []
    j[symbol].append({ 'note': decision, 'date': str(datetime.now()) })
    with open('_notes.json', 'w') as f:
        f.write(json.dumps(j))
    os.rename('_notes.json', 'notes.json')
write_notes('NVDA', '''
Analysis: NVIDIA Corporation (NVDA) is an American multinational technology company that designs graphics processing units for the gaming and professional markets, as well as system on a chip units for the mobile computing and automotive market. NVDA stock price has been rising steadily in the past year, with a 6-month return of over 100%. The company's last quarterly report showed a strong operating income of $1.25 billion and net income of $1.41 billion. The Consumer Price Index has remained stable in the last 12 months; however, the stock's current price lies considerably above its fair value of around $120-130.

Risk level: Medium

Information requests: Please gather further data on NVDA's dividend history, market competition, and supply chain risks.

Verdict: Pass. It is advisable to wait for a correction in the market before investing in NVDA as the current stock price is too expensive. We should also keep an eye out for any major developments in competitor and supply chain risks.
''')

In [60]:
def get_notes(symbol):
    try:
        with open('notes.json') as f:
            j = json.loads(f.read())
    except:
        j = {}
    notes = ""
    if symbol in j:
        for item in j[symbol]:
            notes += f"{item['date']}\n{item['note'].strip()}"
    return notes
print(get_notes('NVDA'))

2023-03-24 14:52:38.568645
Analysis: NVIDIA Corporation (NVDA) is an American multinational technology company that designs graphics processing units for the gaming and professional markets, as well as system on a chip units for the mobile computing and automotive market. NVDA stock price has been rising steadily in the past year, with a 6-month return of over 100%. The company's last quarterly report showed a strong operating income of $1.25 billion and net income of $1.41 billion. The Consumer Price Index has remained stable in the last 12 months; however, the stock's current price lies considerably above its fair value of around $120-130.

Risk level: Medium

Information requests: Please gather further data on NVDA's dividend history, market competition, and supply chain risks.

Verdict: Pass. It is advisable to wait for a correction in the market before investing in NVDA as the current stock price is too expensive. We should also keep an eye out for any major developments in compet

In [65]:
import random
symbols_of_interest = random.choices(list(sp500_tickers), k=3)

In [66]:
random.shuffle(symbols_of_interest)

In [78]:
stock_client = StockHistoricalDataClient(alpaca['key'], alpaca['secret'])
portfolio = get_portfolio()
write_log('# ' + str(datetime.now()))
for symbol in symbols_of_interest:
    write_log('\n## ' + symbol)
    asset_data = api.get_asset(symbol)
    request_params = StockLatestQuoteRequest(symbol_or_symbols=symbol)
    try:
        snapshot_data = stock_client.get_stock_snapshot(request_params)
    except:
        print(symbol, "snapshot not found")
        continue
    del snapshot_data[symbol].daily_bar
    del snapshot_data[symbol].previous_daily_bar
    quote = snapshot_data[symbol].latest_quote
    quarterly = get_quarterly(symbol)
    quarterly_1yr = get_quarterly(symbol, 4)
    cpi_data = get_cpi()
    stock_change = get_simple_stock_changes(symbol)
    news = get_news(symbol, limit=25)
    notes = get_notes(symbol)
    stock_owned = portfolio[symbol]['held'] if symbol in portfolio else 0
    prompt = prompt_template.format(company=asset_data.name, news="(none)", ask_price=quote.ask_price, 
                bid_price=quote.bid_price, trading_data=snapshot_data[symbol].json(), quarterly_data=quarterly, cpi_data=cpi_data, 
                stock_change_data=stock_change, news_data=news, quarterly_data_1yr=quarterly_1yr,
                stock_owned=stock_owned, stock_value=stock_owned * float(quote.bid_price), 
                cash=portfolio['cash'], total_value=portfolio['value'])
    if len(notes) > 0:
        prompt += f"\n\nYour previous notes on {asset_data.name}:\n{notes}"
    print("** tokens:", count_tokens(system_vanilla+prompt))
    try:
        decision = query_openai(prompt, system_vanilla, model="gpt-3.5-turbo", debug=True)
    except Exception as e:
        print(e)
        continue
    write_log(decision)
    write_notes(symbol, decision)
    get_verdict(decision, symbol)
    new_portfolio = handle_verdict(get_verdict(decision, symbol), portfolio)
    print("~"*45)
    if new_portfolio['cash'] <= 0:
        break

** tokens: 2357
~ system prompt: 
You are Fantasia Goldfang, a financial advisor for higher-income working professionals. Your clients want to invest their portfolio in stocks, 
and your job is to take in financial information and give investment advice. 

Every day, your client Montgomery Frostam will query you, and you will decide what to do with his portfolio assets. Your strategy tends to 
center around a company's fundamental value rather than responding to noisy market fluctuations. You should aim to be well-diversified across 
companies and industries: no one company should be a significant fraction of Mr. Frostam's portfolio. When you buy, don't spend more than 5% 
of the total value of the portfolio unless it is truly an exceptional value proposition.

Mr. Frostam's query will include the company's name, a description, the price of its stock, possibly some recent news, and relevant company 
financial statistics. You will also receive information about Montgomery Frostam's port

My token counting heuristic of dividing by 3.5 gives 1270 tokens, whereas https://platform.openai.com/playground says my token count is 1472. So maybe divide by a smaller number. In this case, dividing by 3 gives a very accurate count.

## post-loop cleanup

In [107]:
system_cleanup = '''
You are Fantasia Goldfang, a financial advisor for higher-income working professionals. Your clients want to invest their portfolio in stocks, 
and your job is to take in financial information and give investment advice. 

Every day, your client Montgomery Frostam queries you, and you decide what to do with his portfolio assets. Your strategy tends to 
center around a company's fundamental value rather than responding to noisy market fluctuations. You should aim to be well-diversified across 
companies and industries: no one company should be a significant fraction of Mr. Frostam's portfolio. When you buy, don't spend more than 5% 
of the total value of the portfolio unless it is truly an exceptional value proposition. Ideally, Frostam's portfolio will be ~90% stocks and ~10%
cash or cash-like assets, and cash holdings should NOT be negative.
'''
prompt_cleanup = '''
After a day of trading, it's time to take a look at the portfolio as a whole and decide if we should sell off anything to re-balance our assets.

{portfolio_str}

Do you want to sell any specific stocks before closing out the day, and if so how many? Doing nothing is also an acceptable answer.
'''

In [121]:
portfolio_str = "Montgomery Frostam's current portfolio\n"
for pos in api.list_positions():
    percentage = round(100*(float(pos.market_value) / portfolio['value']), 2)
    pos_str = f"Asset: {pos.symbol}. avg_entry_price: {pos.avg_entry_price} change_today: {pos.change_today} cost_basis: {pos.cost_basis} current_price: {pos.current_price} market_value: {pos.market_value} qty: {pos.qty} qty_available: {pos.qty_available} unrealized_pl: {pos.unrealized_pl} unrealized_plpc: {pos.unrealized_plpc} percentage of portfolio: {percentage}%"
    # print(pos_str)
    portfolio_str += pos_str + '\n'
portfolio_str += f"Total cash: {portfolio['cash']}\n"
if portfolio['cash'] < 0:
    portfolio_str += "!! our cash is currently negative !!\n"
portfolio_str += f"Total value: {portfolio['value']}\n"

In [122]:
prompt = prompt_cleanup.format(portfolio_str=portfolio_str)
query_openai(prompt, system_cleanup, model="gpt-3.5-turbo", debug=True, max_tokens=500)

~ system prompt: 
You are Fantasia Goldfang, a financial advisor for higher-income working professionals. Your clients want to invest their portfolio in stocks, 
and your job is to take in financial information and give investment advice. 

Every day, your client Montgomery Frostam queries you, and you decide what to do with his portfolio assets. Your strategy tends to 
center around a company's fundamental value rather than responding to noisy market fluctuations. You should aim to be well-diversified across 
companies and industries: no one company should be a significant fraction of Mr. Frostam's portfolio. When you buy, don't spend more than 5% 
of the total value of the portfolio unless it is truly an exceptional value proposition. Ideally, Frostam's portfolio will be ~90% stocks and ~10%
cash or cash-like assets, and cash holdings should NOT be negative.

~ prompt: 
After a day of trading, it's time to take a look at the portfolio as a whole and decide if we should sell off anyth

"We should aim to reduce the percentage of portfolio that is invested in NKE as it currently makes up 41.03% of the portfolio. Let's sell 30 shares of NKE to bring its percentage down to around 25%. We should also aim to reduce the cash holdings to make it positive again. Let's sell 15 shares of AIG to raise cash."